# Tutorial 17: Agent Handoffs

In this tutorial, you'll build a **peer-to-peer agent collaboration system** where agents explicitly hand off control to each other.

**What you'll learn:**
- Creating handoff tools that transfer control between agents
- Building peer-to-peer agent workflows (vs supervisor pattern)
- Implementing context preservation across handoffs
- Routing based on explicit agent decisions
- Customer service scenario with sales, support, and billing agents

By the end, you'll have a working handoff system where agents decide when and to whom to transfer conversations.

## Prerequisites

- Completed Tutorials 14-16 (Multi-Agent Patterns)
- Understanding of StateGraph and tool calling
- Familiarity with the supervisor pattern to contrast with handoffs

## Why Agent Handoffs?

Handoffs differ from the supervisor pattern in key ways:

| Pattern | Decision Maker | Structure | Use Case |
|---------|---------------|-----------|----------|
| **Supervisor** | Central supervisor | Hierarchical | Complex tasks needing orchestration |
| **Handoffs** | Individual agents | Peer-to-peer | Service scenarios with clear roles |

**Handoffs are ideal for:**
1. **Customer service**: Sales → Support → Billing
2. **Triage systems**: Intake → Specialist → Follow-up
3. **Clear expertise boundaries**: Each agent knows their limits
4. **Autonomous agents**: Agents make their own routing decisions

```
┌─────────────┐       handoff_to_support        ┌─────────────┐
│   Sales     │────────────────────────────────▶│   Support   │
│   Agent     │                                 │   Agent     │
└─────────────┘                                 └─────────────┘
      ▲                                               │
      │                                               │
      │            handoff_to_billing                 │
      │         ┌─────────────────────────────────────┘
      │         │
      │         ▼
      │    ┌─────────────┐
      └────│   Billing   │
           │   Agent     │
           └─────────────┘
```

## Step 1: Setup and Verify Environment

In [ ]:
from langgraph_ollama_local import LocalAgentConfig
from langchain_ollama import ChatOllama

config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,  # Deterministic for handoff decisions
)

print(f"Model: {config.ollama.model}")
print("Setup complete!")

## Step 2: Define the Handoff State

Our state tracks:
- The current conversation
- Which agent is active
- Handoff target (if agent decides to hand off)
- Context accumulating across agents
- Handoff history for tracking

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
import operator


class HandoffState(TypedDict):
    """State schema for agent handoff pattern."""
    
    # Conversation history
    messages: Annotated[list, add_messages]
    
    # The user's request
    task: str
    
    # Currently active agent
    current_agent: str
    
    # Agent to hand off to (empty if no handoff)
    handoff_target: str
    
    # Shared context accumulating across handoffs
    context: Annotated[list[dict], operator.add]
    
    # Track handoff chain
    handoff_history: Annotated[list[str], operator.add]
    
    # Iteration tracking
    iteration: int
    max_iterations: int
    
    # Final result
    final_result: str


print("State defined!")
print("\nKey differences from supervisor pattern:")
print("- current_agent: Tracks who is active (no central decision maker)")
print("- handoff_target: Agent's own decision to hand off")
print("- handoff_history: Tracks the chain of handoffs")

## Step 3: Create Handoff Tools

The key innovation: **handoff tools** that agents can use to explicitly transfer control.

In [ ]:
from langchain_core.tools import tool
from pydantic import Field


def create_handoff_tool(target_agent: str, description: str):
    """
    Create a tool that allows an agent to hand off to another agent.
    
    This tool signals the intent to transfer control, including
    the reason for the handoff.
    """
    tool_name = f"handoff_to_{target_agent}"
    
    @tool(name=tool_name, description=description)
    def handoff_tool(
        reason: str = Field(
            description="Brief explanation for why you're handing off"
        ),
    ) -> str:
        """Execute the handoff to another agent."""
        return f"Handing off to {target_agent} agent. Reason: {reason}"
    
    return handoff_tool


# Create handoff tools for our customer service scenario
handoff_to_support = create_handoff_tool(
    "support",
    "Transfer to support agent for technical issues, bugs, or how-to questions"
)

handoff_to_billing = create_handoff_tool(
    "billing",
    "Transfer to billing agent for payment, invoice, or pricing questions"
)

handoff_to_sales = create_handoff_tool(
    "sales",
    "Transfer back to sales agent for product questions or new purchases"
)

print("Handoff tools created!")
print(f"\nTool names:")
print(f"  - {handoff_to_support.name}")
print(f"  - {handoff_to_billing.name}")
print(f"  - {handoff_to_sales.name}")

## Step 4: Create Agent Nodes with Handoff Capability

Each agent can:
1. Handle requests in their domain
2. Use handoff tools when requests are outside their expertise

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


AGENT_PROMPT_TEMPLATE = """You are the {agent_name} agent in a customer service team.

Your role: {agent_role}

IMPORTANT INSTRUCTIONS:
1. If you can fully handle the request, provide a complete response and DO NOT use handoff tools
2. Only use a handoff tool if the request is outside your expertise
3. When handing off, explain to the customer that you're transferring them and why

Previous work from other agents:
{previous_context}

Available handoff tools: {handoff_tools}
"""


def create_handoff_agent_node(llm, agent_name: str, agent_role: str, handoff_tools: list):
    """Create an agent node that can handle work or initiate handoffs."""
    
    # Bind tools to LLM
    llm_with_tools = llm.bind_tools(handoff_tools) if handoff_tools else llm
    
    def agent_node(state: HandoffState) -> dict:
        """Execute the agent's work and potentially handoff."""
        
        # Build context from previous agents
        context_items = state.get("context", [])
        if context_items:
            context_parts = []
            for item in context_items:
                agent = item.get("agent", "unknown")
                work = item.get("work", "")
                context_parts.append(f"**{agent}**: {work}")
            previous_context = "\n\n".join(context_parts)
        else:
            previous_context = "This is the first agent handling the request."
        
        # List available handoff tools
        handoff_tools_desc = ", ".join([t.name for t in handoff_tools]) if handoff_tools else "None"
        
        # Build prompt
        system_prompt = AGENT_PROMPT_TEMPLATE.format(
            agent_name=agent_name,
            agent_role=agent_role,
            previous_context=previous_context,
            handoff_tools=handoff_tools_desc,
        )
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=state["task"]),
        ]
        
        # Invoke LLM
        response = llm_with_tools.invoke(messages)
        
        # Check if tool was called (handoff)
        handoff_target = ""
        if hasattr(response, "tool_calls") and response.tool_calls:
            # Agent initiated a handoff
            tool_call = response.tool_calls[0]
            # Extract target from tool name: "handoff_to_<agent>"
            if tool_call["name"].startswith("handoff_to_"):
                handoff_target = tool_call["name"].replace("handoff_to_", "")
        
        # Record this agent's work
        context_entry = {
            "agent": agent_name,
            "work": response.content,
        }
        
        updates = {
            "context": [context_entry],
            "messages": [AIMessage(
                content=f"[{agent_name.title()}] {response.content}",
                tool_calls=response.tool_calls if hasattr(response, "tool_calls") else [],
            )],
        }
        
        # If handoff occurred, record it
        if handoff_target:
            updates["handoff_target"] = handoff_target
            updates["handoff_history"] = [f"{agent_name} -> {handoff_target}"]
        else:
            # No handoff means this agent completed the task
            updates["handoff_target"] = ""
        
        return updates
    
    return agent_node


print("Agent node creator defined!")

## Step 5: Create the Routing Function

Unlike supervisor pattern, routing is based on **agent decisions** (via handoff tools).

In [ ]:
def route_handoffs(state: HandoffState) -> str:
    """
    Route based on handoff decisions.
    
    - If handoff_target is set, route to that agent
    - If no handoff, the agent completed the task -> go to completion
    - If at max iterations, force completion
    """
    handoff_target = state.get("handoff_target", "")
    iteration = state.get("iteration", 0)
    max_iterations = state.get("max_iterations", 10)
    
    # Check iteration limit
    if iteration >= max_iterations:
        print(f"  [Router] Max iterations reached, completing")
        return "complete"
    
    # If there's a handoff target, route to it
    if handoff_target:
        print(f"  [Router] Handing off to {handoff_target}")
        return handoff_target
    
    # No handoff means the current agent completed the task
    print(f"  [Router] Agent completed task, going to completion")
    return "complete"


print("Routing function defined!")
print("\nKey difference from supervisor:")
print("- Routes based on agent's handoff_target (not central decision)")

## Step 6: Create the Completion Node

In [ ]:
def complete_node(state: HandoffState) -> dict:
    """Synthesize final result from all agent work."""
    
    context_items = state.get("context", [])
    
    if not context_items:
        return {
            "final_result": "No agent handled the request.",
            "messages": [AIMessage(content="[System] No response generated.")],
        }
    
    # Build final result from all agent contributions
    parts = []
    for item in context_items:
        agent = item.get("agent", "unknown")
        work = item.get("work", "")
        parts.append(f"**{agent.title()} Agent**:\n{work}")
    
    final_result = "\n\n".join(parts)
    
    return {
        "final_result": final_result,
        "messages": [AIMessage(content="[System] Conversation completed.")],
    }


print("Completion node defined!")

## Step 7: Build the Complete Handoff Graph

Now we assemble everything into a peer-to-peer agent system.

In [ ]:
from langgraph.graph import StateGraph, START, END

# Create agent nodes with their handoff tools
sales_agent = create_handoff_agent_node(
    llm,
    "sales",
    "Handle product inquiries, pricing questions, and sales. You know about our products and can help customers purchase.",
    handoff_tools=[handoff_to_support, handoff_to_billing],
)

support_agent = create_handoff_agent_node(
    llm,
    "support",
    "Handle technical issues, troubleshooting, and how-to questions. You help customers solve problems with products.",
    handoff_tools=[handoff_to_billing, handoff_to_sales],
)

billing_agent = create_handoff_agent_node(
    llm,
    "billing",
    "Handle payment issues, invoices, refunds, and account billing. You manage financial transactions.",
    handoff_tools=[handoff_to_sales, handoff_to_support],
)

# Build the graph
workflow = StateGraph(HandoffState)

# Add agent nodes
workflow.add_node("sales", sales_agent)
workflow.add_node("support", support_agent)
workflow.add_node("billing", billing_agent)
workflow.add_node("complete", complete_node)

# Entry point: start at sales (triage point)
workflow.add_edge(START, "sales")

# Each agent routes based on their handoff decision
for agent_name in ["sales", "support", "billing"]:
    workflow.add_conditional_edges(
        agent_name,
        route_handoffs,
        {
            "sales": "sales",
            "support": "support",
            "billing": "billing",
            "complete": "complete",
        },
    )

# Completion ends the graph
workflow.add_edge("complete", END)

# Compile
graph = workflow.compile()

print("Handoff graph compiled!")
print("\nGraph structure:")
print("  START -> sales (entry point)")
print("  sales -> [sales | support | billing | complete]")
print("  support -> [sales | support | billing | complete]")
print("  billing -> [sales | support | billing | complete]")
print("  complete -> END")

## Step 8: Test the Handoff System

Let's try different customer service scenarios!

In [ ]:
def run_handoff_conversation(task: str, max_iterations: int = 5):
    """Run a conversation through the handoff system."""
    
    print("="*60)
    print(f"Task: {task}")
    print("="*60)
    
    initial_state: HandoffState = {
        "messages": [HumanMessage(content=task)],
        "task": task,
        "current_agent": "sales",
        "handoff_target": "",
        "context": [],
        "handoff_history": [],
        "iteration": 0,
        "max_iterations": max_iterations,
        "final_result": "",
    }
    
    result = graph.invoke(initial_state)
    
    print("\n" + "="*60)
    print("FINAL RESULT")
    print("="*60)
    print(result["final_result"])
    
    if result.get("handoff_history"):
        print("\n" + "="*60)
        print(f"Handoff chain: {' -> '.join(['sales'] + result['handoff_history'])}")
        print(f"Total handoffs: {len(result['handoff_history'])}")
    
    print("="*60)
    
    return result


# Scenario 1: Technical issue (should handoff: sales -> support)
result1 = run_handoff_conversation(
    "My app keeps crashing when I try to export data. How do I fix this?"
)

In [ ]:
# Scenario 2: Billing issue (should handoff: sales -> billing)
result2 = run_handoff_conversation(
    "I was charged twice for my subscription this month. Can you help?"
)

In [ ]:
# Scenario 3: Product question (should stay with sales, no handoff)
result3 = run_handoff_conversation(
    "What features are included in your premium plan?"
)

In [ ]:
# Scenario 4: Complex multi-handoff (sales -> support -> billing)
result4 = run_handoff_conversation(
    "I need help canceling my subscription and getting a refund for this month",
    max_iterations=5
)

## Step 9: Using the Built-in Module

The `langgraph_ollama_local.patterns.handoffs` module provides these utilities:

In [ ]:
from langgraph_ollama_local.patterns.handoffs import (
    create_handoff_graph,
    create_handoff_tool,
    run_handoff_conversation,
)

# Define handoff tools
h_support = create_handoff_tool("support", "Transfer for technical issues")
h_billing = create_handoff_tool("billing", "Transfer for payment questions")
h_sales = create_handoff_tool("sales", "Transfer for product questions")

# Create graph using module
module_graph = create_handoff_graph(
    llm,
    agents={
        "sales": (
            "Handle product questions and sales",
            [h_support, h_billing],
        ),
        "support": (
            "Handle technical issues and troubleshooting",
            [h_billing, h_sales],
        ),
        "billing": (
            "Handle payments and invoices",
            [h_sales, h_support],
        ),
    },
    entry_agent="sales",
    max_iterations=5,
)

# Run conversation using module function
result = run_handoff_conversation(
    module_graph,
    "I need to upgrade my plan and get help with setup",
    entry_agent="sales",
    max_iterations=5,
)

print("Using module utilities!")
print(f"\nFinal result:\n{result['final_result']}")

## Step 10: Visualizing Handoff Patterns

In [ ]:
# Analyze handoff patterns from multiple conversations
test_cases = [
    ("What's your pricing?", "Product question"),
    ("App won't load", "Technical issue"),
    ("Need a refund", "Billing issue"),
    ("Upgrade and invoice help", "Multi-agent need"),
]

print("Handoff Pattern Analysis")
print("="*60)

for task, category in test_cases:
    result = graph.invoke({
        "messages": [HumanMessage(content=task)],
        "task": task,
        "current_agent": "sales",
        "handoff_target": "",
        "context": [],
        "handoff_history": [],
        "iteration": 0,
        "max_iterations": 5,
        "final_result": "",
    })
    
    handoff_chain = "sales"
    if result.get("handoff_history"):
        handoff_chain += " -> " + " -> ".join(
            [h.split(" -> ")[1] for h in result["handoff_history"]]
        )
    
    print(f"\n{category}:")
    print(f"  Task: {task}")
    print(f"  Path: {handoff_chain}")
    print(f"  Agents involved: {len(result['context'])}")

## Complete Code

Here's the complete implementation in one cell for reference:

In [ ]:
# === Complete Agent Handoffs Implementation ===

from typing import Annotated
from typing_extensions import TypedDict
import operator

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import Field

from langgraph_ollama_local import LocalAgentConfig


# === State ===
class HandoffState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    current_agent: str
    handoff_target: str
    context: Annotated[list[dict], operator.add]
    handoff_history: Annotated[list[str], operator.add]
    iteration: int
    max_iterations: int
    final_result: str


# === Handoff Tool ===
def create_handoff_tool(target: str, desc: str):
    @tool(name=f"handoff_to_{target}", description=desc)
    def handoff(reason: str = Field(description="Why handing off")) -> str:
        return f"Handing off to {target}. Reason: {reason}"
    return handoff


# === Agent Node ===
def create_agent(llm, name: str, role: str, tools: list):
    llm_tools = llm.bind_tools(tools) if tools else llm
    
    def agent(state):
        context = "\n".join([f"{c['agent']}: {c['work']}" for c in state.get("context", [])])
        
        response = llm_tools.invoke([
            SystemMessage(content=f"You are {name}. {role}"),
            HumanMessage(content=f"Context: {context}\n\nTask: {state['task']}"),
        ])
        
        handoff = ""
        if hasattr(response, "tool_calls") and response.tool_calls:
            tc = response.tool_calls[0]
            if tc["name"].startswith("handoff_to_"):
                handoff = tc["name"].replace("handoff_to_", "")
        
        updates = {
            "context": [{"agent": name, "work": response.content}],
            "handoff_target": handoff,
        }
        if handoff:
            updates["handoff_history"] = [f"{name} -> {handoff}"]
        
        return updates
    
    return agent


# === Routing ===
def route(state):
    if state["iteration"] >= state["max_iterations"] or not state.get("handoff_target"):
        return "complete"
    return state["handoff_target"]


# === Completion ===
def complete(state):
    parts = [f"{c['agent']}: {c['work']}" for c in state.get("context", [])]
    return {"final_result": "\n\n".join(parts)}


# === Build Graph ===
def build_handoff_graph():
    cfg = LocalAgentConfig()
    llm = ChatOllama(model=cfg.ollama.model, base_url=cfg.ollama.base_url, temperature=0)
    
    # Tools
    h_sup = create_handoff_tool("support", "Tech issues")
    h_bil = create_handoff_tool("billing", "Payment issues")
    h_sal = create_handoff_tool("sales", "Product questions")
    
    # Graph
    g = StateGraph(HandoffState)
    g.add_node("sales", create_agent(llm, "sales", "Handle sales", [h_sup, h_bil]))
    g.add_node("support", create_agent(llm, "support", "Handle tech", [h_bil, h_sal]))
    g.add_node("billing", create_agent(llm, "billing", "Handle payments", [h_sal, h_sup]))
    g.add_node("complete", complete)
    
    g.add_edge(START, "sales")
    for agent in ["sales", "support", "billing"]:
        g.add_conditional_edges(agent, route, 
            {"sales": "sales", "support": "support", "billing": "billing", "complete": "complete"})
    g.add_edge("complete", END)
    
    return g.compile()


# === Use ===
if __name__ == "__main__":
    graph = build_handoff_graph()
    result = graph.invoke({
        "messages": [], "task": "App crashed, need help",
        "current_agent": "sales", "handoff_target": "",
        "context": [], "handoff_history": [],
        "iteration": 0, "max_iterations": 5, "final_result": ""
    })
    print(result["final_result"])
    print("Handoffs:", result["handoff_history"])

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Handoff Tool** | Tool that signals transfer of control to another agent |
| **Peer-to-Peer** | Agents directly hand off to each other (no supervisor) |
| **Context Preservation** | Work accumulates across handoffs for continuity |
| **Explicit Routing** | Agents decide when to hand off based on expertise |
| **Handoff History** | Track the chain of agent transfers |
| **Entry Agent** | First agent to handle all requests (triage point) |
| **Iteration Limit** | Safety mechanism to prevent infinite handoff loops |

## Supervisor vs Handoffs Comparison

| Aspect | Supervisor Pattern | Handoff Pattern |
|--------|-------------------|----------------|
| **Decision Making** | Central supervisor decides routing | Individual agents decide handoffs |
| **Structure** | Hierarchical (supervisor + workers) | Peer-to-peer (equal agents) |
| **Routing Logic** | Supervisor analyzes task + progress | Agent uses tools to signal handoff |
| **Communication** | All through supervisor | Direct agent-to-agent |
| **Best For** | Complex multi-step tasks | Service scenarios with clear roles |
| **Agent Autonomy** | Low (supervisor controls flow) | High (agents control handoffs) |
| **Example Use Case** | Research → Code → Review cycle | Customer service triage |

## When to Use Handoffs

| Use Handoffs When | Use Supervisor When |
|------------------|--------------------|
| Clear role boundaries | Flexible task decomposition |
| Service/triage scenarios | Research/development workflows |
| Agents know their limits | Need orchestration logic |
| Linear handoff chains | Iterative refinement loops |
| Customer-facing flows | Internal task processing |

## Summary

You've learned the fourth multi-agent pattern:

1. **Multi-Agent Collaboration** (Tutorial 14): Supervisor coordinates specialists
2. **Hierarchical Teams** (Tutorial 15): Nested team structures
3. **Subgraph Patterns** (Tutorial 16): Composable, reusable components
4. **Agent Handoffs** (Tutorial 17): Peer-to-peer explicit transfers

**Key Takeaway**: Handoffs give agents autonomy to decide when to transfer control, ideal for service scenarios where each agent knows their expertise boundaries.